# Variational monte carlo simulation for Hydrogen Molecule

In [31]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import fsolve
%matplotlib inline

In [32]:
def wavefunc(r,pars):
    s = pars[0]; a = pars[1]; beta = pars[2];
    r1 = r[:,0:3]
    r2 = r[:,3:6]
    r1L = np.linalg.norm(r1 + np.array([s/2, 0, 0]),axis=1) 
    r1R = np.linalg.norm(r1 - np.array([s/2, 0, 0]),axis=1)
    r2L = np.linalg.norm(r2 + np.array([s/2, 0, 0]),axis=1) 
    r2R = np.linalg.norm(r2 - np.array([s/2, 0, 0]),axis=1)
    r12 = np.linalg.norm(r1-r2,axis=1)
    psi = (np.exp(-r1L/a) + np.exp(-r1R/a)) * (np.exp(-r2L/a) + np.exp(-r2R/a)) * np.exp(r12/(2*(1+beta*r12)))
    return psi
    
def updateWalker(r,N,pars):
    dis = np.random.normal(0,d,(N,6)) #proposed displacement
    p = (wavefunc(r+dis,pars)/wavefunc(r,pars))**2
    rnd = np.random.rand(N)
    r += (rnd<p)[:,None]*dis
    
def EL(r,pars):
    s = pars[0]; a = pars[1]; beta = pars[2];
    r1 = r[:,0:3]
    r2 = r[:,3:6]
    r1L = np.linalg.norm(r1 + np.array([s/2, 0, 0]),axis=1) 
    r1R = np.linalg.norm(r1 - np.array([s/2, 0, 0]),axis=1)
    r2L = np.linalg.norm(r2 + np.array([s/2, 0, 0]),axis=1) 
    r2R = np.linalg.norm(r2 - np.array([s/2, 0, 0]),axis=1)
    r12 = np.linalg.norm(r1-r2,axis=1)
    psi1L = np.exp(-r1L/a)
    psi1R = np.exp(-r1R/a)
    psi2L = np.exp(-r2L/a)
    psi2R = np.exp(-r2R/a)
    psi1 = psi1L+psi1R
    psi2 = psi2L+psi2R
    
    q1L = np.sum((r1 + np.array([s/2, 0, 0]))*(r1-r2),axis=1)/(r1L*r12)  #r1L_hat  dot  r12_hat
    q1R = np.sum((r1 - np.array([s/2, 0, 0]))*(r1-r2),axis=1)/(r1R*r12)  #r1R_hat  dot  r12_hat
    q2L = np.sum((r2 + np.array([s/2, 0, 0]))*(r1-r2),axis=1)/(r2L*r12)  #r2L_hat  dot  r12_hat
    q2R = np.sum((r2 - np.array([s/2, 0, 0]))*(r1-r2),axis=1)/(r2R*r12)  #r2R_hat  dot  r12_hat
    
    Eloc = -1/(a*a) + (psi1L/r1L + psi1R/r1R)/(a*psi1) + (psi2L/r2L + psi2R/r2R)/(a*psi2) - (1/r1L + 1/r1R + 1/r2L + 1/r2R) \
        + 1/r12 + ((psi1L*q1L+psi1R*q1R)/psi1 - (psi2L*q2L+psi2R*q2R)/psi2)/(2*a*(1+beta*r12)**2) \
        - ((4*beta+1)*r12+4)/(4*r12*(1+beta*r12)**4)
    
    return Eloc

def updateVarParams(R,EL,pars,gamma):
    beta = pars[2]
    R12 = np.linalg.norm(R[:,0:3]-R[:,3:6],axis=1)
    dlnpsida = -R12**2 / (2*(1+beta*R12)**2)
    dEda = 2 * (np.mean(EL*dlnpsida)-np.mean(EL)*np.mean(dlnpsida))
    beta -= gamma*dEda
    pars[2] = beta
    return pars, dEda

In [37]:
N = 400 #Number of walkers
T = 26000 #Number of displacement attempts per walker
T0 = 4000 #Reject first T0 timesteps from averaging

#Variational parameters
# alpha = 2 is hard-coded into all functions
s = 1.4
beta = 0.6
impl_cond = lambda a : a*(1+np.exp(-s/a)) - 1
a = fsolve(impl_cond,0.9)[0] #find a from a numerically using the above constraint
pars = [s,a,beta]

d = 1/2 #mean step length
gamma = .9 #damped steepest descent parameter
epsilon = 1e-6 # tolerance on derivative 
itmax = 1; #max number of optimization iterations

dEda = 2; it = 0;
while((abs(dEda) >= epsilon) & (it<itmax)):
    r = np.random.uniform(-s-2*a,s+2*a,(N,6))
    R = np.empty((T*N,6))
    
    for t in range(0,T0):
        updateWalker(r,N,pars)
        
    for t in range(0,T):
        updateWalker(r,N,pars)
        R[t*N:(t+1)*N,:] = r
        
    Eloc = EL(R,pars)
    E = np.mean(Eloc)
    VarE = np.var(Eloc)
    print('pars = ',pars)
    print('<E> = ',E)
    print('Var(EL) = ',VarE)
    pars, dEda = updateVarParams(R,Eloc,pars,gamma)
    print('dEda = ', dEda)
    print('')
    it+=1
    


if it==itmax:
    print('Maximum number of iterations reached. No optimal variational parameter may be found up to the required precision')

pars =  [1.4, 0.84089397653308773, 0.6]
<E> =  -1.86535731928
Var(EL) =  0.05217409125
dEda =  0.000966605728005

Maximum number of iterations reached. No optimal variational parameter may be found up to the required precision
